In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv
/kaggle/input/alx-movie-recommendation-project-2024/movies.csv
/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv
/kaggle/input/alx-movie-recommendation-project-2024/train.csv
/kaggle/input/alx-movie-recommendation-project-2024/test.csv
/kaggle/input/alx-movie-recommendation-project-2024/tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/links.csv


In [2]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

In [3]:
train_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
movies_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/movies.csv')
imdb_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv')
test_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')
links_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/links.csv')
tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/tags.csv')
genome_scores = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv')
sample_submissions = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df.shape

(62423, 3)

In [6]:
sampled_df = train_df.sample(n=400000)

In [7]:
sampled_df.shape

(400000, 4)

In [8]:
train_df.shape

(10000038, 4)

In [9]:
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import Dataset
from surprise import Reader

# Define the Reader with rating scale
reader = Reader(rating_scale=(0.5, 5))

# Load data from DataFrame
data = Dataset.load_from_df(sampled_df[['userId', 'movieId', 'rating']], reader)



In [10]:
from surprise.model_selection import train_test_split, cross_validate
trainset, testset = train_test_split(data, test_size=0.05)

In [11]:
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

# Perform grid search
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

# Use the best parameters to train the final model
algo = gs.best_estimator['rmse']
algo.fit(trainset)

In [12]:
# Initialize the SVD algorithm
#algo = SVD()

# Train the algorithm on the trainset, and evaluate on the testset
#algo.fit(trainset)

# Evaluate the model using cross-validation
#cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=True)

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9219  0.9270  0.9307  0.9281  0.9262  0.9268  0.0029  
Fit time          5.70    5.84    6.02    5.90    5.71    5.83    0.12    
Test time         1.16    0.79    1.18    1.16    0.81    1.02    0.18    


{'test_rmse': array([0.921871  , 0.92695953, 0.93070032, 0.92813422, 0.92624576]),
 'fit_time': (5.69867730140686,
  5.842838525772095,
  6.02089786529541,
  5.897389888763428,
  5.714192867279053),
 'test_time': (1.1600472927093506,
  0.789975643157959,
  1.1757278442382812,
  1.1574304103851318,
  0.8128681182861328)}

In [13]:
# Prepare the test data
testset_for_predictions = [(row['userId'], row['movieId'], 0) for _, row in test_df.iterrows()]

# Predict ratings for the test data
predictions = algo.test(testset_for_predictions)

# Prepare the submission file
submission = pd.DataFrame([(f"{int(pred.uid)}_{int(pred.iid)}", pred.est) for pred in predictions], columns=['Id', 'rating'])


In [14]:
submission.head()

,Id,rating
0,1_2011,3.677069
1,1_4144,4.043634
2,1_5767,3.532027
3,1_6711,3.740250
4,1_7318,3.328615


In [15]:
submission.to_csv('Submission1.csv', index=False)

In [16]:
submission.shape

(5000019, 2)